In [1]:
from theano.sandbox import cuda
cuda.use('gpu2')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(2), GPU number 0 is already in use.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
import numpy as np
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [3]:
batch_size = 64

In [4]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
#Grey images have 1 channel
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

In [6]:
X_train.shape, X_test.shape

((60000, 1, 28, 28), (10000, 1, 28, 28))

In [7]:
def onehot(x):
    return to_categorical(x)

In [8]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [9]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [10]:
def norm_input(x):
    return (x - mean_px)/std_px

# Linear Model

In [11]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model         

In [12]:
lm = get_lin_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [13]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64 )
test_batches = gen.flow(X_test,y_test, batch_size=64)

In [14]:
lm.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n) #batches.n=no of steps per epoch,It should typically be equal to the number of unique samples of your dataset divided by the batch size.

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4226 - acc: 0.8754 - val_loss: 0.3012 - val_acc: 0.9153


In [15]:
lm.optimizer.lr = 0.1

In [16]:
lm.fit_generator(batches,batches.n, nb_epoch=2, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/2
60000/60000 [==============================] - 12s - loss: 0.2985 - acc: 0.9150 - val_loss: 0.2804 - val_acc: 0.9192
Epoch 2/2
60000/60000 [==============================] - 12s - loss: 0.2860 - acc: 0.9196 - val_loss: 0.2789 - val_acc: 0.9230


# Single Dense Layer

In [17]:
def get_fc_model():
    model = Sequential([Lambda(norm_input, input_shape=(1,28,28)),Flatten(), Dense(512, activation='softmax'), Dense(10, activation='softmax')])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
fc = get_fc_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [19]:
fc.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 1.5768 - acc: 0.7701 - val_loss: 1.0864 - val_acc: 0.7643


In [20]:
fc.optimizer.lr = 0.1

In [21]:
fc.fit_generator(batches, batches.n, nb_epoch=4, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.8739 - acc: 0.7787 - val_loss: 0.7196 - val_acc: 0.7837
Epoch 2/4
60000/60000 [==============================] - 13s - loss: 0.6432 - acc: 0.8001 - val_loss: 0.5815 - val_acc: 0.8163
Epoch 3/4
60000/60000 [==============================] - 13s - loss: 0.5181 - acc: 0.8339 - val_loss: 0.4768 - val_acc: 0.8431
Epoch 4/4
60000/60000 [==============================] - 13s - loss: 0.4507 - acc: 0.8476 - val_loss: 0.4406 - val_acc: 0.8506


In [22]:
fc.optimizer.lr=0.01

In [23]:
fc.fit_generator(batches, batches.n, nb_epoch=4, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 13s - loss: 0.4140 - acc: 0.8573 - val_loss: 0.4164 - val_acc: 0.8558
Epoch 2/4
60000/60000 [==============================] - 14s - loss: 0.3905 - acc: 0.8630 - val_loss: 0.4000 - val_acc: 0.8503
Epoch 3/4
60000/60000 [==============================] - 13s - loss: 0.3710 - acc: 0.8665 - val_loss: 0.3863 - val_acc: 0.8545
Epoch 4/4
60000/60000 [==============================] - 13s - loss: 0.3567 - acc: 0.8713 - val_loss: 0.3802 - val_acc: 0.8641


# Basic VGG-Style CNN

In [24]:
def get_model():
    model=Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3,activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512,activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model                        

In [25]:
vg = get_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [26]:
vg.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1066 - acc: 0.9669 - val_loss: 0.0440 - val_acc: 0.9850


In [27]:
vg.optimizer.lr = 0.01

In [28]:
vg.fit_generator(batches, batches.n, nb_epoch=4, validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 21s - loss: 0.0348 - acc: 0.9892 - val_loss: 0.0279 - val_acc: 0.9907
Epoch 2/4
60000/60000 [==============================] - 21s - loss: 0.0226 - acc: 0.9930 - val_loss: 0.0290 - val_acc: 0.9910
Epoch 3/4
60000/60000 [==============================] - 21s - loss: 0.0177 - acc: 0.9943 - val_loss: 0.0302 - val_acc: 0.9905
Epoch 4/4
60000/60000 [==============================] - 21s - loss: 0.0136 - acc: 0.9955 - val_loss: 0.0326 - val_acc: 0.9913


# Data Augmentation


In [29]:
model = get_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [30]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3, 
                           height_shift_range=0.08,zoom_range=0.08)
batches = gen.flow(X_train,y_train,batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [31]:
model.fit_generator(batches,batches.n, nb_epoch=4, 
                   validation_data=test_batches,nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 23s - loss: 0.2000 - acc: 0.9364 - val_loss: 0.0742 - val_acc: 0.9771
Epoch 2/4
60000/60000 [==============================] - 22s - loss: 0.0720 - acc: 0.9782 - val_loss: 0.0562 - val_acc: 0.9826
Epoch 3/4
60000/60000 [==============================] - 23s - loss: 0.0562 - acc: 0.9822 - val_loss: 0.0398 - val_acc: 0.9871
Epoch 4/4
60000/60000 [==============================] - 23s - loss: 0.0469 - acc: 0.9854 - val_loss: 0.0467 - val_acc: 0.9844


In [32]:
model.optimizer.lr=0.01

In [33]:
model.fit_generator(batches, batches.n, nb_epoch=4, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 22s - loss: 0.0418 - acc: 0.9867 - val_loss: 0.0409 - val_acc: 0.9867
Epoch 2/4
60000/60000 [==============================] - 22s - loss: 0.0388 - acc: 0.9878 - val_loss: 0.0427 - val_acc: 0.9864
Epoch 3/4
60000/60000 [==============================] - 22s - loss: 0.0360 - acc: 0.9888 - val_loss: 0.0305 - val_acc: 0.9910
Epoch 4/4
60000/60000 [==============================] - 22s - loss: 0.0322 - acc: 0.9898 - val_loss: 0.0307 - val_acc: 0.9898


In [34]:
model.optimizer.lr = 0.001

In [35]:
model.fit_generator(batches,batches.n, nb_epoch=8, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/8
60000/60000 [==============================] - 22s - loss: 0.0314 - acc: 0.9901 - val_loss: 0.0381 - val_acc: 0.9878
Epoch 2/8
60000/60000 [==============================] - 22s - loss: 0.0291 - acc: 0.9905 - val_loss: 0.0304 - val_acc: 0.9907
Epoch 3/8
60000/60000 [==============================] - 22s - loss: 0.0295 - acc: 0.9907 - val_loss: 0.0310 - val_acc: 0.9912
Epoch 4/8
60000/60000 [==============================] - 22s - loss: 0.0274 - acc: 0.9913 - val_loss: 0.0337 - val_acc: 0.9896
Epoch 5/8
60000/60000 [==============================] - 22s - loss: 0.0256 - acc: 0.9920 - val_loss: 0.0313 - val_acc: 0.9900
Epoch 6/8
60000/60000 [==============================] - 22s - loss: 0.0246 - acc: 0.9921 - val_loss: 0.0233 - val_acc: 0.9930
Epoch 7/8
60000/60000 [==============================] - 22s - loss: 0.0243 - acc: 0.9920 - val_loss: 0.0280 - val_acc: 0.9912
Epoch 8/8
60000/60000 [==============================] - 22s - loss: 0.0241 - acc: 0.9927 - val_loss: 0.0290 - 

In [36]:
model.optimizer.lr = 0.0001

In [37]:
model.fit_generator(batches,batches.n, nb_epoch=10, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/10
60000/60000 [==============================] - 22s - loss: 0.0218 - acc: 0.9931 - val_loss: 0.0290 - val_acc: 0.9898
Epoch 2/10
60000/60000 [==============================] - 22s - loss: 0.0220 - acc: 0.9926 - val_loss: 0.0299 - val_acc: 0.9908
Epoch 3/10
60000/60000 [==============================] - 22s - loss: 0.0221 - acc: 0.9930 - val_loss: 0.0281 - val_acc: 0.9913
Epoch 4/10
60000/60000 [==============================] - 22s - loss: 0.0206 - acc: 0.9934 - val_loss: 0.0236 - val_acc: 0.9922
Epoch 5/10
60000/60000 [==============================] - 22s - loss: 0.0207 - acc: 0.9933 - val_loss: 0.0298 - val_acc: 0.9906
Epoch 6/10
60000/60000 [==============================] - 21s - loss: 0.0192 - acc: 0.9938 - val_loss: 0.0267 - val_acc: 0.9922
Epoch 7/10
60000/60000 [==============================] - 22s - loss: 0.0198 - acc: 0.9939 - val_loss: 0.0278 - val_acc: 0.9914
Epoch 8/10
60000/60000 [==============================] - 22s - loss: 0.0201 - acc: 0.9940 - val_loss: 0

# Batch Norm + Data Augmentation

In [38]:
def get_model_bn():
    model=Sequential([
        Lambda(norm_input,input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(axis=1),
        Dense(512, activation='relu'),
        BatchNormalization(axis=1),
        Dense(10, activation='relu'),
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [39]:
model = get_model_bn()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [40]:
model.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 38s - loss: nan - acc: 0.1960 - val_loss: nan - val_acc: 0.0972


In [41]:
model.optimizer.lr = 0.1

In [42]:
model.fit_generator(batches, batches.n, nb_epoch=4, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0984 - val_loss: nan - val_acc: 0.0963
Epoch 2/4
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.1022
Epoch 3/4
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0988 - val_loss: nan - val_acc: 0.0927
Epoch 4/4
60000/60000 [==============================] - 39s - loss: nan - acc: 0.0982 - val_loss: nan - val_acc: 0.1014


In [43]:
model.optimizer.lr = 0.01

In [44]:
model.fit_generator(batches, batches.n, nb_epoch=10, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0993 - val_loss: nan - val_acc: 0.0955
Epoch 2/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0980 - val_loss: nan - val_acc: 0.0993
Epoch 3/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0995 - val_loss: nan - val_acc: 0.0979
Epoch 4/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0988 - val_loss: nan - val_acc: 0.0971
Epoch 5/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0989 - val_loss: nan - val_acc: 0.0999
Epoch 6/10
60000/60000 [==============================] - 39s - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.0996
Epoch 7/10
60000/60000 [==============================] - 39s - loss: nan - acc: 0.0984 - val_loss: nan - val_acc: 0.0953
Epoch 8/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0983 - val_loss: nan - val_acc: 0.0983
Epoch 9/10
60000/60000 [

In [45]:
model.optimizer.lr = 0.001

In [46]:
model.fit_generator(batches, batches.n, nb_epoch=10, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0990 - val_loss: nan - val_acc: 0.0953
Epoch 2/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0978 - val_loss: nan - val_acc: 0.0993
Epoch 3/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0995 - val_loss: nan - val_acc: 0.1022
Epoch 4/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0980 - val_loss: nan - val_acc: 0.0960
Epoch 5/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0987 - val_loss: nan - val_acc: 0.0968
Epoch 6/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0991 - val_loss: nan - val_acc: 0.0954
Epoch 7/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0993 - val_loss: nan - val_acc: 0.0998
Epoch 8/10
60000/60000 [==============================] - 38s - loss: nan - acc: 0.0986 - val_loss: nan - val_acc: 0.0983
Epoch 9/10
60000/60000 [

# Batchnorm+Data Augmentation+Dropout

In [47]:
def get_model_bn_do():
    model=Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [48]:
model = get_model_bn_do()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [49]:
model.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 40s - loss: 0.2270 - acc: 0.9313 - val_loss: 0.0842 - val_acc: 0.9738


In [50]:
model.optimizer.lr=0.1

In [51]:
model.fit_generator(batches, batches.n, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/4
60000/60000 [==============================] - 39s - loss: 0.0932 - acc: 0.9712 - val_loss: 0.0622 - val_acc: 0.9791
Epoch 2/4
60000/60000 [==============================] - 39s - loss: 0.0778 - acc: 0.9767 - val_loss: 0.0526 - val_acc: 0.9832
Epoch 3/4
60000/60000 [==============================] - 39s - loss: 0.0668 - acc: 0.9796 - val_loss: 0.0457 - val_acc: 0.9860
Epoch 4/4
60000/60000 [==============================] - 39s - loss: 0.0621 - acc: 0.9810 - val_loss: 0.0417 - val_acc: 0.9866


In [52]:
model.optimizer.lr=0.01


In [53]:
model.fit_generator(batches, batches.n, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/12
60000/60000 [==============================] - 39s - loss: 0.0561 - acc: 0.9824 - val_loss: 0.0307 - val_acc: 0.9906
Epoch 2/12
60000/60000 [==============================] - 40s - loss: 0.0537 - acc: 0.9834 - val_loss: 0.0297 - val_acc: 0.9912
Epoch 3/12
60000/60000 [==============================] - 39s - loss: 0.0511 - acc: 0.9845 - val_loss: 0.0316 - val_acc: 0.9891
Epoch 4/12
60000/60000 [==============================] - 39s - loss: 0.0495 - acc: 0.9857 - val_loss: 0.0276 - val_acc: 0.9906
Epoch 5/12
60000/60000 [==============================] - 39s - loss: 0.0430 - acc: 0.9869 - val_loss: 0.0324 - val_acc: 0.9899
Epoch 6/12
60000/60000 [==============================] - 39s - loss: 0.0449 - acc: 0.9860 - val_loss: 0.0361 - val_acc: 0.9883
Epoch 7/12
60000/60000 [==============================] - 40s - loss: 0.0429 - acc: 0.9870 - val_loss: 0.0306 - val_acc: 0.9905
Epoch 8/12
60000/60000 [==============================] - 39s - loss: 0.0385 - acc: 0.9883 - val_loss: 0

In [54]:
model.optimizer.lr=0.001


In [55]:
model.fit_generator(batches,batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
60000/60000 [==============================] - 39s - loss: 0.0349 - acc: 0.9893 - val_loss: 0.0218 - val_acc: 0.9917


# Ensembling

In [56]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches, batches.n, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.1
    model.fit_generator(batches, batches.n, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.01
    model.fit_generator(batches, batches.n, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.001
    model.fit_generator(batches, batches.n, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    return model

In [57]:
models = [fit_model() for i in range(6)]


/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_8 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_9 and cannot be automatically inferred with the Thean

In [62]:
path = "MNIST_WEIGHTS/"
model_path = path + 'models/'

In [63]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [64]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])


 9472/10000 [===========================>..] - ETA: 0s

In [65]:
evals.mean(axis=0)

array([ 0.013 ,  0.9959])

In [66]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])


In [67]:
all_preds.shape


(4, 10000, 10)

In [68]:

avg_preds = all_preds.mean(axis=0)

In [69]:

keras.metrics.categorical_accuracy(y_test, avg_preds).eval()

array(0.9965999722480774, dtype=float32)